## Setup and Imports

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

In [ ]:
checkpoint = "meta-llama/Llama-3.2-3B-Instruct"
model = AutoModelForCausalLM.from_pretrained(checkpoint, torch_dtype=torch.bfloat16, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

model = model.eval()

config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [ ]:
def generate(prompt, model, tokenizer):
    inputs = tokenizer(
        prompt,
        return_tensors="pt",
    ).to(model.device)

    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=150, do_sample=True)

    result = outputs[0][inputs["input_ids"].shape[-1]:]
    return tokenizer.decode(result, skip_special_tokens=False)

## Custom Tool

In [ ]:
def add_two_integers(x: int, y: int):
    """
    Adds two integers.

    Args:
        x: An integer
        y: An integer
    """
    return x + y

In [ ]:
prior_system_instruction = """\
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are an expert in composing functions. You are given a question and a set of possible functions.
Based on the question, you will need to make one or more function/tool calls to achieve the purpose.
If none of the function can be used, point it out. If the given question lacks the parameters required by the function,
also point it out. You should only return the function call in tools call sections.

If you decide to invoke any of the function(s), you MUST put it in the format of [func_name1(params_name1=params_value1, params_name2=params_value2...), func_name2(params)]
You SHOULD NOT include any other text in the response.

Here is a list of functions in JSON format that you can invoke.\n\n"""

custom_tool = """\
[
    {
        "name": "add_two_integers",
        "description": "Adds two integer numerals",
        "parameters": {
            "type": "dict",
            "required": ["x", "y"],
            "properties": {
                "x": {
                    "type": "integer",
                    "description": "An integer"
                },
                "y": {
                    "type": "integer",
                    "description": "An integer"
                },
            }
        }
    }
]"""

system_instruction = "\nYou are a helpful assistant"

system_prompt = f"{prior_system_instruction}{custom_tool}{system_instruction}<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n"

In [ ]:
user_prompt = "What is the result of 12322 added to 1242453"

prompt = f"{system_prompt}{user_prompt}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"

In [ ]:
print(prompt)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are an expert in composing functions. You are given a question and a set of possible functions.
Based on the question, you will need to make one or more function/tool calls to achieve the purpose.
If none of the function can be used, point it out. If the given question lacks the parameters required by the function,
also point it out. You should only return the function call in tools call sections.

If you decide to invoke any of the function(s), you MUST put it in the format of [func_name1(params_name1=params_value1, params_name2=params_value2...), func_name2(params)]
You SHOULD NOT include any other text in the response.

Here is a list of functions in JSON format that you can invoke.

[
    {
        "name": "add_two_integers",
        "description": "Adds two integer numerals",
        "parameters": {
            "type": "dict",
            "required": ["x", "y"],
            "properties": {
                "x": {
    

In [ ]:
tool_call_response = generate(prompt, model, tokenizer)
print(tool_call_response)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[add_two_integers(x=12322, y=1242453)]<|eot_id|>


## Call to the tool

In [ ]:
out = str(add_two_integers(x=12322, y=1242453))
print(out)

1254775


## Add everything to the main prompt

In [ ]:
prompt = f"{prompt}<|python_tag|>{tool_call_response}<|start_header_id|>ipython<|end_header_id|>\n\n"

prompt = prompt + f'"\\"{out}\\""<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n'
print(prompt)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are an expert in composing functions. You are given a question and a set of possible functions.
Based on the question, you will need to make one or more function/tool calls to achieve the purpose.
If none of the function can be used, point it out. If the given question lacks the parameters required by the function,
also point it out. You should only return the function call in tools call sections.

If you decide to invoke any of the function(s), you MUST put it in the format of [func_name1(params_name1=params_value1, params_name2=params_value2...), func_name2(params)]
You SHOULD NOT include any other text in the response.

Here is a list of functions in JSON format that you can invoke.

[
    {
        "name": "add_two_integers",
        "description": "Adds two integer numerals",
        "parameters": {
            "type": "dict",
            "required": ["x", "y"],
            "properties": {
                "x": {
    

In [ ]:
print(generate(prompt, model, tokenizer))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


The result of the addition is 1254775.<|eot_id|>


## Chat Template

In [ ]:
chat_template="""\
{{- bos_token }}
{%- if custom_tools is defined %}
    {%- set tools = custom_tools %}
{%- endif %}
{%- if not tools_in_user_message is defined %}
    {%- set tools_in_user_message = false %}
{%- endif %}
{%- if not date_string is defined %}
    {%- if strftime_now is defined %}
        {%- set date_string = strftime_now("%d %b %Y") %}
    {%- else %}
        {%- set date_string = "24 September 2024" %}
    {%- endif %}
{%- endif %}
{%- if not tools is defined %}
    {%- set tools = none %}
{%- endif %}

{#- This block extracts the system message, so we can slot it into the right place. #}
{%- if messages[0]['role'] == 'system' %}
    {%- set system_message = messages[0]['content']|trim %}
    {%- set messages = messages[1:] %}
{%- else %}
    {%- set system_message = "" %}
{%- endif %}

{#- System message + environment setup (modified for 3.2) #}
{{- "<|start_header_id|>system<|end_header_id|>\n\n" }}
{%- if tools is not none %}
    {{- "You are an expert in composing functions. You are given a question and a set of possible functions.\n" }}
    {{- "Based on the question, you will need to make one or more function/tool calls to achieve the purpose.\n" }}
    {{- "If none of the functions can be used, point it out. If the given question lacks the parameters required by the function,\n" }}
    {{- "also point it out. You should only return the function call in the tools call sections.\n\n" }}
    {{- "If you decide to invoke any of the function(s), you MUST put it in the format of [func_name1(params_name1=params_value1, params_name2=params_value2...), func_name2(params)]\n" }}
    {{- "You SHOULD NOT include any other text in the response.\n\n" }}
    {{- "Here is a list of functions in JSON format that you can invoke.\n\n" }}
    {{- "[\n" }}
    {%- for t in tools %}
        {%- set tool_json = t['function'] | tojson(indent=4) %}
        {{- "    " + tool_json | replace('\n', '\n    ') }}
        {{- ",\n" if not loop.last else "\n" }}
    {%- endfor %}
    {{- "]\n" }}
{%- elif builtin_tools is defined %}
    {{- "Environment: ipython\n" }}
    {{- "Tools: " + builtin_tools | reject('equalto', 'code_interpreter') | join(", ") }}
    {{- "Cutting Knowledge Date: December 2023\n" }}
    {{- "Today Date: " + date_string + "\n\n" }}
{%- endif %}
{{- system_message }}
{{- "<|eot_id|>" }}

{#- Custom tools passed in a user message (modified for consistency with 3.2) #}
{%- if tools_in_user_message and tools is not none %}
    {#- Extract the first user message to include tools if necessary #}
    {%- if messages | length != 0 %}
        {%- set first_user_message = messages[0]['content']|trim %}
        {%- set messages = messages[1:] %}
    {%- else %}
        {{- raise_exception("Cannot put tools in the first user message when there's no first user message!") }}
    {%- endif %}
    {{- "<|start_header_id|>user<|end_header_id|>\n\n" }}
    {{- "You are an expert in composing functions. You are given a question and a set of possible functions.\n" }}
    {{- "Based on the question, you will need to make one or more function/tool calls to achieve the purpose.\n" }}
    {{- "If none of the functions can be used, point it out. If the given question lacks the parameters required by the function,\n" }}
    {{- "also point it out. You should only return the function call in the tools call sections.\n\n" }}
    {{- "If you decide to invoke any of the function(s), you MUST put it in the format of [func_name1(params_name1=params_value1, params_name2=params_value2...), func_name2(params)]\n" }}
    {{- "You SHOULD NOT include any other text in the response.\n\n" }}
    {{- "Here is a list of functions in JSON format that you can invoke.\n\n" }}
    {{- "[\n" }}
    {%- for t in tools %}
        {%- set tool_json = t['function'] | tojson(indent=4) %}
        {{- "    " + tool_json | replace('\n', '\n    ') }}
        {{- ",\n" if not loop.last else "\n" }}
    {%- endfor %}
    {{- "]\n" }}
    {{- first_user_message + "<|eot_id|>"}}
{%- endif %}

{#- Loop through messages to handle user, assistant, and tool interactions -#}
{%- for message in messages %}
    {%- if message.role == 'user' %}
        {{- '<|start_header_id|>user<|end_header_id|>\n\n' + message['content'] | trim + '<|eot_id|>' }}
    {%- elif message.role == 'assistant' and 'tool_calls' in message %}
        {#- Handle zero-shot tool calls (modified for 3.2) #}
        {{- '<|start_header_id|>assistant<|end_header_id|>\n\n' }}
        {%- for tool_call in message.tool_calls %}
            {{- '<|python_tag|>[' + tool_call.function.name + '(' }}
            {%- for arg_name, arg_val in tool_call.function.arguments.items() %}
                {{- arg_name + '=' + arg_val}}
                {%- if not loop.last %}
                    {{- ", " }}
                {%- endif %}
            {%- endfor %}
            {{- ')]' }}
        {%- endfor %}
        {{- '<|eot_id|>' }}
    {%- elif message.role == 'assistant' %}
        {{- '<|start_header_id|>assistant<|end_header_id|>\n\n' + message['content'] | trim + '<|eot_id|>' }}
    {%- elif message.role == 'tool' %}
        {#- Modified for 3.2 #}
        {{- "<|start_header_id|>ipython<|end_header_id|>\n\n" }}
        {%- if message.content is mapping or message.content is iterable %}
            {{- message.content | tojson }}
        {%- else %}
            {{- message.content }}
        {%- endif %}
        {{- "<|eot_id|>" }}
    {%- elif message.role == 'code' %}
        {#- Code interpreter handling (maintained from 3.1) #}
        {{- "<|python_tag|>" + message['content'] | trim + "<|eom_id|>" }}
    {%- endif %}
{%- endfor %}

{#- Add the prompt for generation if specified -#}
{%- if add_generation_prompt %}
    {{- '<|start_header_id|>assistant<|end_header_id|>\n\n' }}
{%- endif %}\
"""

In [ ]:
tokenizer.chat_template = chat_template

In [ ]:
messages = [
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": "What is the result of 12322 added to 1242453"},
    {"role": "assistant", "tool_calls": [{"type": "function", "function": {"name": "add_two_integers", "arguments": {"x": "1234", "y": "1242453"}}}]},
    {"role": "tool", "name": "add_two_integers", "content": "\"1254775\""},
]
prompt = tokenizer.apply_chat_template(
    messages,
    tools=[add_two_integers],
    tokenize=False,
    add_generation_prompt=True,
)

print(prompt)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are an expert in composing functions. You are given a question and a set of possible functions.
Based on the question, you will need to make one or more function/tool calls to achieve the purpose.
If none of the functions can be used, point it out. If the given question lacks the parameters required by the function,
also point it out. You should only return the function call in the tools call sections.

If you decide to invoke any of the function(s), you MUST put it in the format of [func_name1(params_name1=params_value1, params_name2=params_value2...), func_name2(params)]
You SHOULD NOT include any other text in the response.

Here is a list of functions in JSON format that you can invoke.

[
    {
        "name": "add_two_integers",
        "description": "Adds two integers.",
        "parameters": {
            "type": "object",
            "properties": {
                "x": {
                    "type": "integer",
 

In [ ]:
print(generate(prompt, model, tokenizer))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


The result of adding 12322 to 1242453 is 1254775.<|eot_id|>
